# Multi-Document Agents

In this notebook we will look into Building RAG when you have a large number of documents using `DocumentAgents` concept with `ReAct Agent`.

### Installation

In [ ]:
!pip install llama-index
!pip install llama-index-llms-anthropic
!pip install llama-index-embeddings-huggingface

### Set Logging

In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

# Set up the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set logger level to INFO

# Clear out any existing handlers
logger.handlers = []

# Set up the StreamHandler to output to sys.stdout (Colab's output)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to INFO

# Add the handler to the logger
logger.addHandler(handler)

from IPython.display import display, HTML

### Set Anthropic API Key

In [2]:
import os
os.environ['ANTHROPIC_API_KEY'] = 'YOUR ANTHROPIC API KEY'

### Set LLM and Embedding model

We will use anthropic latest released `Claude-3 Opus` LLM.

In [3]:
from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [4]:
llm = Anthropic(temperature=0.0, model='claude-3-opus-20240229')
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [5]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

### Download Documents

We will use Wikipedia pages of `Toronto`, `Seattle`, `Chicago`, `Boston`, `Houston` cities and build RAG pipeline.

In [6]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Houston"]

from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

### Load Document

In [7]:
# Load all wiki documents

from llama_index.core import SimpleDirectoryReader

city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

#### Build ReAct Agent for each city 

In [8]:
from llama_index.core.agent import ReActAgent
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# Build agents dictionary
agents = {}

for wiki_title in wiki_titles:
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        city_docs[wiki_title],
    )
    # build summary index
    summary_index = SummaryIndex.from_documents(
        city_docs[wiki_title],
    )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine()

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"Useful for retrieving specific context from {wiki_title}"
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for summarization questions related to"
                    f" {wiki_title}"
                ),
            ),
        ),
    ]

    # build agent
    agent = ReActAgent.from_tools(
        query_engine_tools,
        llm=llm,
        verbose=True,
    )

    agents[wiki_title] = agent

#### Define IndexNode for each of these Agents

In [9]:
from llama_index.core.schema import IndexNode

# define top-level nodes
objects = []
for wiki_title in wiki_titles:
    # define index node that links to these agents
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. Use"
        " this index if you need to lookup specific facts about"
        f" {wiki_title}.\nDo not use this index if you want to analyze"
        " multiple cities."
    )
    node = IndexNode(
        text=wiki_summary, index_id=wiki_title, obj=agents[wiki_title]
    )
    objects.append(node)

#### Define Top-Level Retriever to choose an Agent

In [10]:
vector_index = VectorStoreIndex(
    objects=objects,
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)

#### Test Queries

Should choose a vector tool/ summary tool for a specific agent based on the query.

In [11]:
# should use Toronto agent -> vector tool
response = query_engine.query("What is the population of Toronto?")

Retrieval entering Toronto: ReActAgent
Retrieving from object ReActAgent with query What is the population of Toronto?
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me answer the question.
Action: vector_tool
Action Input: {'input': 'What is the population of Toronto?'}
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Observation: According to the context information, the population of Toronto in 2021 was 2,794,356, making it the fourth-most populous city in North America.
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: I can answer without using any more tools.
Answer: According to the information provided, the population of Toronto in 2021 was 2,794,356, making it the fourth-most populous city in North America.
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


In [12]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [13]:
# should use Houston agent -> vector tool
response = query_engine.query("Who and when was Houston founded?")

Retrieval entering Houston: ReActAgent
Retrieving from object ReActAgent with query Who and when was Houston founded?
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me answer the question about who founded Houston and when it was founded.
Action: vector_tool
Action Input: {'input': 'Who founded Houston and when was it founded?'}
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Observation: Houston was founded by land investors on August 30, 1836, at the confluence of Buffalo Bayou and White Oak Bayou, a point now known as Allen's Landing. The city was incorporated on June 5, 1837 and named after former General Sam Houston, who was president of the Republic of Texas at the time.
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: The vector_tool provided the key information needed to answer the question of who founded Houston and when it was founded. I can now prov

In [14]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [15]:
# should use Boston agent -> summary tool
response = query_engine.query("Summarize about the sports teams in Boston")

Retrieval entering Boston: ReActAgent
Retrieving from object ReActAgent with query Summarize about the sports teams in Boston
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me answer the question.
Action: summary_tool
Action Input: {'input': 'Summarize the sports teams in Boston'}
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Observation: Boston has teams in the four major North American men's professional sports leagues plus Major League Soccer, and has won 39 championships in these leagues:

- The Boston Red Sox (MLB) play at Fenway Park. They are one of the most storied franchises in baseball.

- The Boston Celtics (NBA) play at TD Garden. Along with the Los Angeles Lakers, they have won the most NBA championships with 17. 

- The Boston Bruins (NHL) also play at TD Garden. They were the first American NHL team and are an Original Six franchise.

- The New England Patriots (NFL) play in ne

In [16]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [17]:
# should use Seattle agent -> summary tool
response = query_engine.query(
    "Give me a summary on all the positive aspects of Chicago"
)

Retrieval entering Chicago: ReActAgent
Retrieving from object ReActAgent with query Give me a summary on all the positive aspects of Chicago
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me summarize the positive aspects of Chicago.
Action: summary_tool
Action Input: {'input': 'Provide a summary of the positive aspects and attributes of the city of Chicago'}
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Observation: Based on the provided information, some of the positive aspects and attributes of Chicago include:

- Chicago is the third most populous city in the U.S. and a major transportation hub. It is an important component in global distribution.

- The city has a diverse economy, with major industries including finance, commerce, industry, technology, telecommunications, and transportation. Chicago has the second largest central business district in the U.S. 

- Chicago is an internatio

In [18]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))